# Reporte de uso de MP para gestion de reposiscion

In [1]:
import MySQLdb
from datetime import datetime, timedelta
import pandas as pd

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()

ModuleNotFoundError: No module named 'MySQLdb'

In [2]:
pip install mysqlclient

  Using cached mysqlclient-2.2.7.tar.gz (91 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
      Trying pkg-config --exists mysqlclient
      Command 'pkg-config --exists mysqlclient' returned non-zero exit status 1.
      Trying pkg-config --exists mariadb
      Command 'pkg-config --exists mariadb' returned non-zero exit status 1.
      Trying pkg-config --exists libmariadb
      Command 'pkg-config --exists libmariadb' returned non-zero exit status 1.
      Trying pkg-config --exists perconaserverclient
      Command 'pkg-config --exists perconaserverclient' returned non-zero exit status 1.
      Traceback (most recent call last):
        File "/home/hugojz/GIT/myvenv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main(

In [ ]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [ ]:
fecha_ayer = datetime.now()
fecha_ayer = fecha_ayer.strftime('%Y-%m-%d')

In [ ]:
cur.execute("""
    SELECT 
        productox.Codigo AS Codigo, 
        productox.Nombre AS Producto, 
        SUM(dcaptura.Valor) AS Dosificado 
    FROM 
        dbp8100.dcaptura AS dcaptura
    JOIN 
        dbp8100.tareaseje AS tareaseje ON dcaptura.IDT = tareaseje.NroID
    JOIN 
        dbp8100.productox AS productox ON productox.NroID = dcaptura.IDP 
    WHERE 
        tareaseje.Fecha >= %s AND 
        tareaseje.Fecha <= %s 
    GROUP BY 
        productox.Nombre
""", (fecha_ayer, fecha_actual))

In [ ]:
# Obtener los resultados
resultados = cur.fetchall()
print(resultados)